# Hubble Source Catalog SWEEPS Proper Motion Notebook
### August 2019, Steve Lubow and Rick White

A [new MAST interface](https://catalogs.mast.stsci.edu/hsc) supports queries to the current and previous versions of the [Hubble Source Catalog](https://archive.stsci.edu/hst/hsc). It allows searches of the summary table (with multi-filter mean photometry) and the detailed table (with all the multi-epoch measurements).  It also has an associated [API](https://catalogs.mast.stsci.edu/docs/hsc.html), which is used in this notebook.

The [web-based user interface](https://catalogs.mast.stsci.edu/hsc) to the HSC does not currently include access to the new proper motions available for the [SWEEPS](https://media.stsci.edu/news_release/news/2011-16) field in version 3.1 of the [Hubble Source Catalog](https://archive.stsci.edu/hst/hsc). However those tables are accessible via the API. This notebook shows how to use them.

This notebook is similar to the [previously released version](../sweeps/sweeps_hscv3p1.html) that uses CasJobs rather than the new API.  The Casjobs interface is definitely more powerful and flexible, but the API is simpler to use for users who are not already experienced Casjobs users.  Currently the API does not include easy access to the colors and magnitudes of the SWEEPS objects, but they will be added soon.

Additional information is available on the 
[SWEEPS Proper Motions help page](https://archive.stsci.edu/hst/hsc/help/sweeps/hsc_sweeps_pm.html).

This notebook is available for [download](sweeps_hscv3p1_api.ipynb).  A simpler notebook that provides a quick introduction to the HSC API is also [available](hscv3_api.html).  Another [simple notebook](hscv3_smc_api.html) generates a color-magnitude diagram for the Small Magellanic Cloud in only a couple of minutes.

# Instructions: 
* Complete the initialization steps [described below](#Initialization).
* Run the notebook to completion. 
* Modify and rerun any sections of the Table of Contents below.

Running the notebook from top to bottom takes about 
4 minutes (depending on the speed of your computer).


# Table of Contents
* [Initialization](#Initialization)
* [Properties of Full Catalog](#fullcat)
    * [Sky Coverage](#SkyCoverage)
    * [Proper Motion Distributions](#pmhist)
    * [Visit Distribution](#visitshist)
    * [Time Coverage Distributions](#timehist)
    * [Detection Positions](#detpos)
    * [Positions for a Sample With Good PMs](#good_sample)
* [Science Applications](#sciap)
    * [High Proper Motion Objects](#hpm)
    * [HLA Cutout Images for Selected Objects](#cutouts)

# Initialization <a class="anchor" id="Initialization"></a>

### Install Python modules

_This notebook requires the use of **Python 3**._

This needs some special modules in addition to the common requirements of `astropy`, `numpy` and `scipy`.  For anaconda versions of Python the installation commands are:

<pre>
conda install requests
conda install pillow
pip install fastkde
</pre>

#### Other optional configuration

If desired, you can set `resPath`, the output directory, in the next code block (the default location is the current working directory, which is probably the same directory as this script).

In [ ]:
resPath="./" # directory where generated plots are saved

%matplotlib inline
import astropy, pylab, time, sys, os, requests, json
import numpy as np
from matplotlib.colors import LogNorm

## For handling ordinary astropy Tables
from astropy.table import Table
from astropy.io import fits, ascii

from PIL import Image
from io import BytesIO

from fastkde import fastKDE
from scipy.interpolate import RectBivariateSpline
from astropy.modeling import models, fitting

# There are a number of relatively unimportant warnings that 
# show up, so for now, suppress them:
import warnings
warnings.filterwarnings("ignore")

# Set page width to fill browser for longer output lines
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
# set width for pprint
astropy.conf.max_width = 150

## Useful functions

Execute HSC searches and resolve names using [MAST query](https://mast.stsci.edu/api/v0/MastApiTutorial.html).

In [ ]:
hscapiurl = "https://catalogs.mast.stsci.edu/api/v0.1/hsc"

def hsccone(ra,dec,radius,table="summary",release="v3",format="csv",magtype="magaper2",
            columns=None, baseurl=hscapiurl, verbose=False,
            **kw):
    """Do a cone search of the HSC catalog
    
    Parameters
    ----------
    ra (float): (degrees) J2000 Right Ascension
    dec (float): (degrees) J2000 Declination
    radius (float): (degrees) Search radius (<= 0.5 degrees)
    table (string): summary, detailed, propermotions, or sourcepositions
    release (string): v3 or v2
    magtype (string): magaper2 or magauto (only applies to summary table)
    format: csv, votable, json
    columns: list of column names to include (None means use defaults)
    baseurl: base URL for the request
    verbose: print info about request
    **kw: other parameters (e.g., 'numimages.gte':2)
    """
    
    data = kw.copy()
    data['ra'] = ra
    data['dec'] = dec
    data['radius'] = radius
    return hscsearch(table=table,release=release,format=format,magtype=magtype,
                     columns=columns,baseurl=baseurl,verbose=verbose,**data)


def hscsearch(table="summary",release="v3",magtype="magaper2",format="csv",
              columns=None, baseurl=hscapiurl, verbose=False,
           **kw):
    """Do a general search of the HSC catalog (possibly without ra/dec/radius)
    
    Parameters
    ----------
    table (string): summary, detailed, propermotions, or sourcepositions
    release (string): v3 or v2
    magtype (string): magaper2 or magauto (only applies to summary table)
    format: csv, votable, json
    columns: list of column names to include (None means use defaults)
    baseurl: base URL for the request
    verbose: print info about request
    **kw: other parameters (e.g., 'numimages.gte':2).  Note this is required!
    """
    
    data = kw.copy()
    if not data:
        raise ValueError("You must specify some parameters for search")
    if format not in ("csv","votable","json"):
        raise ValueError("Bad value for format")
    url = "{}.{}".format(cat2url(table,release,magtype,baseurl=baseurl),format)
    if columns:
        # check that column values are legal
        # create a dictionary to speed this up
        dcols = {}
        for col in hscmetadata(table,release,magtype)['name']:
            dcols[col.lower()] = 1
        badcols = []
        for col in columns:
            if col.lower().strip() not in dcols:
                badcols.append(col)
        if badcols:
            raise ValueError('Some columns not found in table: {}'.format(', '.join(badcols)))
        # two different ways to specify a list of column values in the API
        # data['columns'] = columns
        data['columns'] = '[{}]'.format(','.join(columns))

    # either get or post works
    # r = requests.post(url, data=data)
    r = requests.get(url, params=data)

    if verbose:
        print(r.url)
    r.raise_for_status()
    if format == "json":
        return r.json()
    else:
        return r.text


def hscmetadata(table="summary",release="v3",magtype="magaper2",baseurl=hscapiurl):
    """Return metadata for the specified catalog and table
    
    Parameters
    ----------
    table (string): summary, detailed, propermotions, or sourcepositions
    release (string): v3 or v2
    magtype (string): magaper2 or magauto (only applies to summary table)
    baseurl: base URL for the request
    
    Returns an astropy table with columns name, type, description
    """
    url = "{}/metadata".format(cat2url(table,release,magtype,baseurl=baseurl))
    r = requests.get(url)
    r.raise_for_status()
    v = r.json()
    # convert to astropy table
    tab = Table(rows=[(x['name'],x['type'],x['description']) for x in v],
               names=('name','type','description'))
    return tab


def cat2url(table="summary",release="v3",magtype="magaper2",baseurl=hscapiurl):
    """Return URL for the specified catalog and table
    
    Parameters
    ----------
    table (string): summary, detailed, propermotions, or sourcepositions
    release (string): v3 or v2
    magtype (string): magaper2 or magauto (only applies to summary table)
    baseurl: base URL for the request
    
    Returns a string with the base URL for this request
    """
    checklegal(table,release,magtype)
    if table == "summary":
        url = "{baseurl}/{release}/{table}/{magtype}".format(**locals())
    else:
        url = "{baseurl}/{release}/{table}".format(**locals())
    return url


def checklegal(table,release,magtype):
    """Checks if this combination of table, release and magtype is acceptable
    
    Raises a ValueError exception if there is problem
    """
    
    releaselist = ("v2", "v3")
    if release not in releaselist:
        raise ValueError("Bad value for release (must be one of {})".format(
            ', '.join(releaselist)))
    if release=="v2":
        tablelist = ("summary", "detailed")
    else:
        tablelist = ("summary", "detailed", "propermotions", "sourcepositions")
    if table not in tablelist:
        raise ValueError("Bad value for table (for {} must be one of {})".format(
            release, ", ".join(tablelist)))
    if table == "summary":
        magtypelist = ("magaper2", "magauto")
        if magtype not in magtypelist:
            raise ValueError("Bad value for magtype (must be one of {})".format(
                ", ".join(magtypelist)))


def mastQuery(request, url='https://mast.stsci.edu/api/v0/invoke'):
    """Perform a MAST query.

    Parameters
    ----------
    request (dictionary): The MAST request json object
    url (string): The service URL

    Returns the returned data content
    """
    
    # Encoding the request as a json string
    requestString = json.dumps(request)
    r = requests.post(url, data={'request': requestString})
    r.raise_for_status()
    return r.text


def resolve(name):
    """Get the RA and Dec for an object using the MAST name resolver
    
    Parameters
    ----------
    name (str): Name of object

    Returns RA, Dec tuple with position
    """

    resolverRequest = {'service':'Mast.Name.Lookup',
                       'params':{'input':name,
                                 'format':'json'
                                },
                      }
    resolvedObjectString = mastQuery(resolverRequest)
    resolvedObject = json.loads(resolvedObjectString)
    # The resolver returns a variety of information about the resolved object, 
    # however for our purposes all we need are the RA and Dec
    try:
        objRa = resolvedObject['resolvedCoordinate'][0]['ra']
        objDec = resolvedObject['resolvedCoordinate'][0]['decl']
    except IndexError as e:
        raise ValueError("Unknown object '{}'".format(name))
    return (objRa, objDec)

### Use metadata query to get information on available columns

This query works for any of the tables in the API (summary, detailed, propermotions, sourcepositions).

In [ ]:
meta = hscmetadata("propermotions")
print(' '.join(meta['name']))
meta[:5]

### Retrieve data on selected SWEEPS objects

This makes a single large request to the HSC search interface to the get the contents of the proper motions table.  Despite its large size (460K rows), the query is relatively efficient: it takes about 25 seconds to retrieve the results from the server, and then another 20 seconds to convert it to an astropy table.  The speed of the table conversion will depend on your computer.

Note that the query restricts the sample to objects with at least 20 images total spread over at least 10 different visits.
These constraints can be modified depending on your science goals.

In [ ]:
columns = """ObjID,raMean,decMean,raMeanErr,decMeanErr,NumFilters,NumVisits,
    pmLat,pmLon,pmLatErr,pmLonErr,pmLatDev,pmLonDev,epochMean,epochStart,epochEnd""".split(",")
columns = [x.strip() for x in columns]
columns = [x for x in columns if x and not x.startswith('#')]

# missing -- impossible with current data I think
# MagMed, n, MagMAD

constraints = {'NumFilters.gt':1, 'NumVisits.gt':9, 'NumImages.gt':19}

# note the pagesize parameter, which allows retrieving very large results
# the default pagesize is 50000 rows

t0 = time.time()
results = hscsearch(table="propermotions",release='v3',columns=columns,verbose=True,
                    pagesize=500000, **constraints)
print("{:.1f} s: retrieved data".format(time.time()-t0))
tab = ascii.read(results)
print("{:.1f} s: converted to astropy table".format(time.time()-t0))

# change some column names for consistency with the Casjobs version of this notebook

tab.rename_column("raMean","RA")
tab.rename_column("decMean","Dec")
tab.rename_column("raMeanErr","RAerr")
tab.rename_column("decMeanErr","Decerr")
tab.rename_column("pmLat","bpm")
tab.rename_column("pmLon","lpm")
tab.rename_column("pmLatErr","bpmerr")
tab.rename_column("pmLonErr","lpmerr")

# compute some additional columns

tab['pmdev'] = np.sqrt(tab['pmLonDev']**2+tab['pmLatDev']**2)
tab['yr'] = (tab['epochMean'] - 47892)/365.25+1990
tab['dT'] = (tab['epochEnd']-tab['epochStart'])/365.25
tab['yrStart'] = (tab['epochStart'] - 47892)/365.25+1990
tab['yrEnd'] = (tab['epochEnd'] - 47892)/365.25+1990

# delete some columns that are not needed after the computations
del tab['pmLonDev'], tab['pmLatDev'], tab['epochEnd'], tab['epochStart'], tab['epochMean']

tab

# Properties of Full Catalog  <a class="anchor" id="fullcat"></a>

## Sky Coverage  <a class="anchor" id="SkyCoverage"></a>

In [ ]:
x = tab['RA']
y = tab['Dec']

pylab.rcParams.update({'font.size': 16})
pylab.figure(1,(12,10))
pylab.scatter(x, y, s=1)
pylab.autoscale(tight=True)
pylab.xlabel('RA')
pylab.ylabel('Dec')
dc=0.01
pylab.xlim(min(x)-dc, max(x)+dc)
pylab.ylim(min(y)-dc, max(y)+dc)
pylab.gca().invert_xaxis()
pylab.text(0.5,0.93,'{:,} stars in SWEEPS'.format(len(x)),
       horizontalalignment='left',
       transform=pylab.gca().transAxes)

# Proper Motion Histograms <a class="anchor" id="pmhist"></a>

## Proper motion histograms for lon and lat 

In [ ]:
bin = 0.2
hrange = (-20,20)
bincount = int((hrange[1]-hrange[0])/bin + 0.5) + 1
pylab.rcParams.update({'font.size': 16})
pylab.figure(1,(12,10))
pylab.hist(tab['lpm'], range=hrange, bins=bincount, label='Longitude', 
           histtype='step', linewidth=2)
pylab.hist(tab['bpm'], range=hrange, bins=bincount, label='Latitude', 
           histtype='step', linewidth=2)
pylab.xlabel('Proper motion [mas/yr]')
pylab.ylabel('Number [in {:.2} mas bins]'.format(bin))
pylab.legend(loc='upper right')
pylab.autoscale(enable=True, axis='x', tight=True)
pylab.ylim(0,13500)
pylab.title('{:,} stars in SWEEPS'.format(len(tab)))
pylab.tight_layout()
pylab.savefig('{}sweeps_api_pmerr_hist.png'.format(resPath))

## Proper motion error cumulative histogram for lon and lat

In [ ]:
bin = 0.01
hrange = (0,2)
bincount = int((hrange[1]-hrange[0])/bin + 0.5) + 1
pylab.rcParams.update({'font.size': 16})
pylab.figure(1,(12,10))
pylab.hist(tab['lpmerr'], range=hrange, bins=bincount, label='Longitude Error', 
           histtype='step', cumulative=1, linewidth=2)
pylab.hist(tab['bpmerr'], range=hrange, bins=bincount, label='Latitude Error', 
           histtype='step', cumulative=1, linewidth=2)
pylab.xlabel('Proper motion error [mas/yr]')
pylab.ylabel('Cumulative number [in {:0.2} mas bins]'.format(bin))
pylab.legend(loc='upper right')
pylab.autoscale(enable=True, axis='x', tight=True)
pylab.ylim(0,500000)
pylab.title('{:,} stars in SWEEPS'.format(len(tab)))
pylab.tight_layout()
pylab.savefig('{}sweeps_api_pmerr_cumhist.png'.format(resPath))

## Proper motion error log histogram for lon and lat 

In [ ]:
bin = 0.01
hrange = (0,6)
bincount = int((hrange[1]-hrange[0])/bin + 0.5) + 1
pylab.rcParams.update({'font.size': 16})
pylab.figure(1,(12,10))
pylab.hist(tab['lpmerr'], range=hrange, bins=bincount, label='Longitude Error', 
           histtype='step', linewidth=2)
pylab.hist(tab['bpmerr'], range=hrange, bins=bincount, label='Latitude Error', 
           histtype='step', linewidth=2)
pylab.xlabel('Proper motion error [mas/yr]')
pylab.ylabel('Number [in {:0.2} mas bins]'.format(bin))
pylab.legend(loc='upper right')
pylab.yscale('log')
pylab.autoscale(enable=True, axis='x', tight=True)
pylab.ylim(0,15000)
pylab.title('{:,} stars in SWEEPS'.format(len(tab)))
pylab.tight_layout()
pylab.savefig('{}sweeps_api_pmerr_loghist.png'.format(resPath))

## Proper motion error as a function of dT

Exclude objects with dT near zero, and to improve the plotting add a bit of random noise to spread out the quanitized time values.

In [ ]:
# restrict to sources with dT > 1 year
dtmin = 1.0
w = np.where(tab['dT']>dtmin)[0]
if ('rw' not in locals()) or len(rw) != len(w):
    rw = np.random.random(len(w))
x = np.array(tab['dT'][w]) + 0.5*(rw-0.5)
y = np.log(np.array(tab['lpmerr'][w]))

# Calculate the point density
t0 = time.time()
myPDF,axes = fastKDE.pdf(x,y,numPoints=2**9+1)
print("kde took {:.1f} sec".format(time.time()-t0))

# interpolate to get z values at points
finterp = RectBivariateSpline(axes[1],axes[0],myPDF)
z = finterp(y,x,grid=False)

# Sort the points by density, so that the densest points are plotted last
idx = z.argsort()
xs, ys, zs = x[idx], y[idx], z[idx]

# select a random subset of points in the most crowded regions to speed up plotting
wran = np.where(np.random.random(len(zs))*zs<0.05)[0]
print("Plotting {} of {} points".format(len(wran),len(zs)))
xs = xs[wran]
ys = ys[wran]
zs = zs[wran]

pylab.rcParams.update({'font.size': 16})
pylab.figure(1,(12,10))
pylab.yscale('log')
pylab.scatter(xs, np.exp(ys), c=zs, s=2, edgecolor='', cmap='plasma', 
              label='Longitude PM error')
pylab.autoscale(tight=True, axis='y')
pylab.xlim(0.0, max(x)*1.05)
pylab.xlabel('Date range [yr]')
pylab.ylabel('Proper motion error [mas/yr]')
pylab.legend(loc='best')
pylab.title('{:,} stars in SWEEPS'.format(len(tab)))
pylab.colorbar()
pylab.tight_layout()
pylab.savefig('{}sweeps_api_pmerr_vs_dt.png'.format(resPath))

## Proper motion error log histogram for lon and lat 

Divide sample into points with $<6$ years of data and points with more than 6 years of data.

In [ ]:
bin = 0.01
hrange = (0,6)
bincount = int((hrange[1]-hrange[0])/bin + 0.5) + 1

tsplit = 6

pylab.rcParams.update({'font.size': 16})
pylab.figure(1,(12,12))
pylab.subplot(211)
w = np.where(tab['dT']<=tsplit)[0]
pylab.hist(tab['lpmerr'][w], range=hrange, bins=bincount, label='Longitude Error', 
           histtype='step', linewidth=2)
pylab.hist(tab['bpmerr'][w], range=hrange, bins=bincount, label='Latitude Error', 
           histtype='step', linewidth=2)
pylab.xlabel('Proper motion error [mas/yr]')
pylab.ylabel('Number [in {:0.2} mas bins]'.format(bin))
pylab.legend(loc='upper right')
pylab.yscale('log')
pylab.autoscale(enable=True, axis='x', tight=True)
pylab.ylim(0,15000)
pylab.title('{:,} stars in SWEEPS with dT < {} yrs'.format(len(w),tsplit))
pylab.tight_layout()

pylab.subplot(212)
w = np.where(tab['dT']>tsplit)
pylab.hist(tab['lpmerr'][w], range=hrange, bins=bincount, label='Longitude Error', 
           histtype='step', linewidth=2)
pylab.hist(tab['bpmerr'][w], range=hrange, bins=bincount, label='Latitude Error', 
           histtype='step', linewidth=2)
pylab.xlabel('Proper motion error [mas/yr]')
pylab.ylabel('Number [in {:0.2} mas bins]'.format(bin))
pylab.legend(loc='upper right')
pylab.yscale('log')
pylab.autoscale(enable=True, axis='x', tight=True)
pylab.ylim(0,15000)
pylab.title('{:,} stars in SWEEPS with dT > {} yrs'.format(len(w),tsplit))
pylab.tight_layout()

pylab.savefig('{}sweeps_api_pmerr_loghist2.png'.format(resPath))

# Number of Visits Histogram <a class="anchor" id="visitshist"></a>

In [ ]:
bin = 1
hrange = (0,130)
bincount = int((hrange[1]-hrange[0])/bin + 0.5) + 1
pylab.rcParams.update({'font.size': 16})
pylab.figure(1,(12,10))
pylab.hist(tab['NumVisits'], range=hrange, bins=bincount, label='Number of visits ', 
           histtype='step', linewidth=2)
pylab.xlabel('Number of visits')
pylab.ylabel('Number of objects')
pylab.autoscale(enable=True, axis='x', tight=True)
pylab.ylim(0,200000)
pylab.title('{:,} stars in SWEEPS'.format(len(tab)))
pylab.tight_layout()
pylab.savefig('{}sweeps_api_numvisits_hist.png'.format(resPath))

# Time Histograms <a class="anchor" id="timehist"></a>

First plot histogram of observation dates.

In [ ]:
bin = 1
hrange = (2000, 2020)
bincount = int((hrange[1]-hrange[0])/bin + 0.5) + 1
pylab.rcParams.update({'font.size': 16})
pylab.figure(1,(12,10))
pylab.hist(tab['yr'], range=hrange, bins=bincount, label='year ', histtype='step', linewidth=2)
pylab.xlabel('mean detection epoch (year)')
pylab.ylabel('Number of objects')
pylab.autoscale(enable=True, axis='x', tight=True)
pylab.ylim(0,300000)
pylab.title('{:,} stars in SWEEPS'.format(len(tab)))
pylab.tight_layout()
pylab.savefig('{}sweeps_api_year_hist.png'.format(resPath))

Then plot histogram of observation duration for the objects.

In [ ]:
bin = 0.25
hrange = (0, 15)
bincount = int((hrange[1]-hrange[0])/bin + 0.5) + 1
pylab.rcParams.update({'font.size': 16})
pylab.figure(1,(12,10))
pylab.hist(tab['dT'], range=hrange, bins=bincount, label='year ', histtype='step', linewidth=2)
pylab.xlabel('time span (years)')
pylab.ylabel('Number of objects')
pylab.autoscale(enable=True, axis='x', tight=True)
pylab.yscale('log')
pylab.title('{:,} stars in SWEEPS'.format(len(tab)))
pylab.tight_layout()
pylab.savefig('{}sweeps_api_year_hist.png'.format(resPath))

# Detection Positions <a class="anchor" id="detpos"></a>

Define a function to plot the PM fit for an object.

In [ ]:
# define function
def positions(Obj):
    """
    input parameter Obj is the value of the ObjID 
    output plots change in (lon, lat) as a function of time
    overplots proper motion fit
    provides ObjID and proper motion information in labels
    """

    # get the measured positions as a function of time
    pos = ascii.read(hscsearch("sourcepositions", columns="dT,dLon,dLat".split(','), objid=Obj))
    pos.sort('dT')
    
    # get the PM fit parameters
    pm = ascii.read(hscsearch("propermotions", columns="pmlon,pmlonerr,pmlat,pmlaterr".split(','), objid=Obj))
    
    lpm = pm['pmlon'][0]
    bpm = pm['pmlat'][0]
    
    # get the intercept for the proper motion fit referenced to the start time
    # time between mean epoch and zero (ref) epoch (years)

    x = pos['dT']
    y = pos['dLon']
    pylab.rcParams.update({'font.size':10})
    pylab.figure(1,(6,3))
    pylab.subplot(121)
    pylab.scatter(x, y, s=10)
    # xpm = np.linspace(0, max(x), 10)
    xpm = np.array([x.min(),x.max()])
    ypm = lpm*xpm
    pylab.plot(xpm, ypm, '-r')
    pylab.xlabel('dT (yrs)')
    pylab.ylabel('dLon (mas)')
    y = pos['dLat']
    pylab.subplot(122)
    pylab.scatter(x, y, s=10)
    ypm = bpm*xpm
    pylab.plot(xpm, ypm, '-r')
    pylab.xlabel('dT (yrs)')
    pylab.ylabel('dLat (mas)')
    pylab.suptitle("""ObjID {}
{} detections,  (lpm, bpm) = ({:.1f}, {:.1f}) mas/yr""".format(Obj, len(x), lpm, bpm),
                  size=10)
    pylab.tight_layout(rect=[0, 0.0, 1, 0.88])
    pylab.show()
    pylab.close()

## Plot positions for a sample of objects with good proper motions  <a class="anchor" id="good_sample"></a>

This selects objects that are detected in more than 90 visits with a median absolute deviation from the fit of less than 1.5 mas and proper motion error less than 1.0 mas/yr.

In [ ]:
n = tab['NumVisits']
dev = tab['pmdev']
objid = tab['ObjID']
lpmerr0 = np.array(tab['lpmerr'])
bpmerr0 = np.array(tab['bpmerr'])
wi = np.where( (dev < 1.5) & (n > 90) & (np.sqrt(bpmerr0**2+lpmerr0**2) < 1.0))[0]
print("Plotting {} objects".format(len(wi)))
for o in objid[wi]:
    positions(o)

# Science Applications <a class="anchor" id="sciap"></a>

## High Proper Motion Objects  <a class="anchor" id="hpm"></a>

Get a list of objects with high, accurately measured proper motions.
Proper motions are measured relative to the Galactic center.

In [ ]:
lpm_sgra = -6.379 # +- 0.026
bpm_sgra = -0.202 # +- 0.019

lpm0 = np.array(tab['lpm'])
bpm0 = np.array(tab['bpm'])
lpmerr0 = np.array(tab['lpmerr'])
bpmerr0 = np.array(tab['bpmerr'])
pmtot0 = np.sqrt((bpm0-bpm_sgra)**2+(lpm0-lpm_sgra)**2)
pmerr0 = np.sqrt(bpmerr0**2+lpmerr0**2)
dev = tab['pmdev']

# sort samples by decreasing PM
wpmh = np.where((pmtot0 > 15) & (pmerr0 < 1.0) & (dev < 5))[0]
wpmh = wpmh[np.argsort(-pmtot0[wpmh])]

print("Plotting {} objects".format(len(wpmh)))
for o in tab["ObjID"][wpmh]:
    positions(o)

## Get HLA cutout images for selected objects  <a class="anchor" id="cutouts"></a>

Get HLA color cutout images for the high-PM objects.  The `query_hla` function gets a table of all the color images that are available at a given position using the f814w+f606w filters.  The `get_image` function reads a single cutout image (as a JPEG color image) and returns a PIL image object.

See the documentation on [HLA VO services](http://hla.stsci.edu/hla_help.html#services) and the [fitscut image cutout service](http://hla.stsci.edu/fitscutcgi_interface.html) for more information on the web services being used.

In [ ]:
def query_hla(ra,dec,size=0.0,imagetype="color",inst="ACS",format="image/jpeg",
              spectral_elt=("f814w","f606w"),autoscale=95.0,asinh=1,
              naxis=33):
    # convert a list of filters to a comma-separated string
    if not isinstance(spectral_elt,str):
        spectral_elt = ",".join(spectral_elt)
    siapurl = ("https://hla.stsci.edu/cgi-bin/hlaSIAP.cgi?"
               "pos={ra},{dec}&size={size}&imagetype={imagetype}&inst={inst}"
               "&format={format}&spectral_elt={spectral_elt}"
               "&autoscale={autoscale}&asinh={asinh}"
               "&naxis={naxis}").format(**locals())
    votable = Table.read(siapurl,format="votable")
    return votable

def get_image(url):
    
    """Get image from a URL"""
    
    r = requests.get(url)
    im = Image.open(BytesIO(r.content))
    return im

# display earliest and latest images side-by-side
# top 10 highest PM objects
wsel = wpmh[:10]
nim = len(wsel)
icols = 1        # objects per row
ncols = 2*icols  # two images for each object
nrows = (nim+icols-1)//icols

imsize = 33
xcross = np.array([-1,1,0,0,0])*2 + imsize/2
ycross = np.array([0,0,0,-1,1])*2 + imsize/2

pylab.rcParams.update({"font.size":14})
pylab.figure(1,(12, (12/ncols)*nrows))
for jim, i in enumerate(wsel):
    hlatab = query_hla(tab["RA"][i],tab["Dec"][i],naxis=imsize)
    # sort by observation date
    hlatab = hlatab[np.argsort(hlatab['StartTime'])]
    k = 0
    im1 = get_image(hlatab['URL'][k])
    pylab.subplot(nrows,ncols,2*jim+1)
    pylab.imshow(im1,origin="upper")
    pylab.plot(xcross,ycross,'g')
    pylab.title(hlatab['StartTime'][k].decode('utf-8'))
    pylab.ylabel("ObjID {}".format(tab["ObjID"][i]))

    k = -1
    im2 = get_image(hlatab['URL'][k])
    pylab.subplot(nrows,ncols,2*jim+2)
    pylab.imshow(im2,origin="upper")
    pylab.plot(xcross,ycross,'g')
    pylab.title(hlatab['StartTime'][k].decode('utf-8'))

## Look at the entire collection of images for the highest PM object

In [ ]:
i = wpmh[0]
print(tab['ObjID','RA','Dec','bpm','lpm','yr','dT'][i])
imsize = 33
hlatab = query_hla(tab["RA"][i],tab["Dec"][i],naxis=imsize)
# sort by observation date
hlatab = hlatab[np.argsort(hlatab['StartTime'])]

nim = len(hlatab)
ncols = 8
nrows = (nim+ncols-1)//ncols

xcross = np.array([-1,1,0,0,0])*2 + imsize/2
ycross = np.array([0,0,0,-1,1])*2 + imsize/2

pylab.rcParams.update({"font.size":11})
pylab.figure(1,(20, (20/ncols)*nrows))
t0 = time.time()
for k in range(nim):
    im1 = get_image(hlatab['URL'][k])
    pylab.subplot(nrows,ncols,k+1)
    pylab.imshow(im1,origin="upper")
    pylab.plot(xcross,ycross,'g')
    pylab.title(hlatab['StartTime'][k].decode('utf-8'))
    if ((k+1) % 10)==0:
        print("{:.1f} s: finished {} of {}".format(time.time()-t0,k+1,nim))
pylab.tight_layout()
print("{:.1f} s: finished {}".format(time.time()-t0,nim))